In [1]:
import os

from utils.auxiliary_func import *
from flim_analysis.spatial_analysis.spatial_func import *
import tifffile
from tqdm import tqdm
import pandas as pd
import config.const as const

### Define Parameters

In [3]:
patch_size = 1500
overlap = 0.75
fold_method='stratified_kfold'


DATA_PREPARATION_DIR = const.DATA_PREPARATION_DIR

### Calculate probability for each nucleus by aggregation of the patch probabilities

In [4]:
patch_df = read_patches_files(patch_size, overlap)

In [5]:
fold_method='stratified_kfold'
hist_base_dir = os.path.join(const.DISTRIBUTION_RESULTS_PATCH_DIR, f"overlap_{patch_size}", fold_method)
best_hist_aggregated_df_1500 = process_hist_results(patch_df, hist_base_dir,seed_type='best')

mean_auc :0.6569855072463768
best_auc: 0.7753623188405797
median_auc: 0.6572463768115941
Seed corresponding to best AUC: 3006
Number of rows with NaN values: 0
Aggregated DataFrame shape: (7603771, 7)


In [6]:
best_hist_aggregated_df_1500.head()

,nucleus_label,leap_ID,lifetime_mean,X coordinate,Y coordinate,category,prob_results
0,1,015,4.114694,3662.775510,1900.285714,non responder,0.510318
1,1,017,4.067597,2863.145161,3150.403226,non responder,0.636030
2,1,019,4.099839,3291.403226,1459.112903,non responder,0.775295
3,1,021,4.631462,2815.384615,2049.615385,responder,0.242873
4,1,023,4.368353,2224.588235,1628.117647,responder,0.489741


In [7]:
file_path = os.path.join(DATA_PREPARATION_DIR, f'mean_prob_patch_size_{patch_size}_for_each_nucleus.csv')
best_hist_aggregated_df_1500.to_csv(file_path, index=False)

### Create flim probability image

In [8]:
file_path = os.path.join(DATA_PREPARATION_DIR, f'mean_prob_patch_size_{patch_size}_for_each_nucleus.csv')
flim_classification_probabilities = pd.read_csv(file_path, dtype={'leap_ID': str})
flim_classification_probabilities.head()

,nucleus_label,leap_ID,lifetime_mean,X coordinate,Y coordinate,category,prob_results
0,1,015,4.114694,3662.775510,1900.285714,non responder,0.510318
1,1,017,4.067597,2863.145161,3150.403226,non responder,0.636030
2,1,019,4.099839,3291.403226,1459.112903,non responder,0.775295
3,1,021,4.631462,2815.384615,2049.615385,responder,0.242873
4,1,023,4.368353,2224.588235,1628.117647,responder,0.489741


In [6]:
for leap_id in tqdm(flim_classification_probabilities["leap_ID"].unique()):
    
    flim_single_nuclear_seg = tifffile.imread(
        os.path.join(const.SEG_AFTER_QC_DIR, f"LEAP{leap_id}_segmentation_labels_qc.tif")
    ).astype(np.int32)

    # Filter probabilities
    leap_probs = flim_classification_probabilities[
        flim_classification_probabilities["leap_ID"] == leap_id
    ].copy()

    max_label = flim_single_nuclear_seg.max()
    lookup = np.full(
        max_label + 1, np.nan, dtype=leap_probs["prob_results"].dtype
    )
    lookup[leap_probs["nucleus_label"].values] = leap_probs["prob_results"].values
    flim_single_nuclear_probabilities = lookup[flim_single_nuclear_seg]


    im_tif = Image.fromarray(flim_single_nuclear_probabilities)

    os.makedirs(const.FLIM_MODEL_PROBABILITY_DIR, exist_ok=True)

    flim_prob_path = os.path.join(const.FLIM_MODEL_PROBABILITY_DIR, f'Leap{leap_id}_flim_probability.tif')

    tifffile.imwrite(flim_prob_path, flim_single_nuclear_probabilities.astype(np.float32))

100%|██████████| 53/53 [07:31<00:00,  8.51s/it]
